In [47]:
import math
import time

# all 8 directions
dx = [-1, -1, -1, 0, 1, 1,  1,  0]
dy = [-1,  0,  1, 1, 1, 0, -1, -1]

def in_bounds(x, y, R, C):
    return (0 <= x and x < R and 0 <= y and y < C)
  
# recursive backtracking to determine if str can be found in board
def helper1(R, C, board, visited, x, y, str, index):
    # if found all letters, then return true
    if index == len(str):
        return True

    # traverse in a new direction
    for i in range(8):
        x1 = x + dx[i]
        y1 = y + dy[i]
        # if new direction is valid and is equal to the next letter, recurse
        if in_bounds(x1, y1, R, C) and not visited[x1][y1] and board[x1][y1] == str[index]:
            visited[x1][y1] = True
            temp = helper1(R, C, board, visited, x1, y1, str, index + 1)
            visited[x1][y1] = False
            if temp:
                return True
    
# O(RC)
# determines if str can be found in board
def in_board(board, str):
    R = len(board)
    C = len(board[0])
    visited = []
    for i in range(R):
        temp = []
        for j in range(C):
            temp.append(False)
        visited.append(temp)

    # finds the position of the first letter before recursing on helper method
    for i in range(R):
        for j in range(C):
            if board[i][j] == str[0]:
                visited[i][j] = True
                temp = helper1(R, C, board, visited, i, j, str, 1)
                visited[i][j] = False
                if temp:
                    return True

    return False

def get_word_from_file(lens, name = "allWords.txt"):
    with open(name, "r") as f:
        for line in f.readlines():
            # get rid of \n at each line
            line = line.rstrip("\n")
            word = line.upper()
            if len(word) in lens:
                yield word

def prune_list(board, lens):
    my_set = set()
    R = len(board)
    C = len(board[0])
    
    for i in range(R):
        for j in range(C):
            for k in range(8):
                i1 = i + dx[k]
                j1 = j + dy[k]
                if in_bounds(i1, j1, R, C):
                    for m in range(8):
                        i2 = i1 + dx[m]
                        j2 = j1 + dy[m]
                        if (not (i2 == i and j2 == j) and in_bounds(i2, j2, R, C)):
                            if board[i][j] != ' ' and board[i1][j1] != ' ' and board[i2][j2] != ' ':
                                my_set.add(board[i][j] + board[i1][j1] + board[i2][j2])
      
    ans = []
    
    for word in get_word_from_file(lens):
        valid = True
        for j in range(2, len(word)):
            key = word[j-2] + word[j-1] + word[j]
            if key not in my_set:
                valid = False
                break

        if valid:
            ans.append(word)

    return ans

def find_word(board, word, word_to_location):
    visited = []
    for i in range(len(board)):
        visited.append([False] * len(board))
    for r,row in enumerate(board):
        for c,char in enumerate(row):
            if (char is word[0]):
                visited[r][c] = True
                coord_list = []
                coord_list.append((r, c))
                check_word(board, word, word_to_location, r, c, visited, 1, coord_list)
                visited[r][c] = False
            
            
            
            
def check_word(board, word, word_to_location, r, c, visited, index, cur_list):
    if(index >= len(word)):
        if not word in word_to_location:
            word_to_location[word] = []
        new_list = []
        for coord in cur_list:
            new_list.append(coord)
        word_to_location[word].append(new_list)
        return
    
    for i in range(8):
        new_r = r + dy[i]
        new_c = c + dx[i]
        if in_bounds(new_r, new_c, len(board), len(board)):
            if not visited[new_r][new_c]:
                if board[new_r][new_c] is word[index]:
                    visited[new_r][new_c] = True
                    cur_list.append((new_r, new_c))
                    check_word(board, word, word_to_location, new_r, new_c, visited, index + 1, cur_list)
                    cur_list.remove((new_r, new_c))
                    visited[new_r][new_c] = False

def get_length_map(word_to_location):
    length_to_words = {}
    for word in word_to_location:
        if not len(word) in length_to_words:
            length_to_words[len(word)] = []
        length_to_words[len(word)].append(word)
    return length_to_words

def frequency_pruning(freqs, lens, length_to_words, index, pruned_list, cur_words, length_to_index):
    if (index >= len(lens)):
        new_words = []
        for word in cur_words:
            new_words.append(word)
        pruned_list.append(new_words)
        return
    for i in range(length_to_index[lens[index]], len(length_to_words[lens[index]])):
        word = length_to_words[lens[index]][i]
        valid = True
        for char in word:
            freqs[ord(char) - ord('A')] -= 1
        for char in word:
            if freqs[ord(char) - ord('A')] < 0:
                valid = False
                break
        if valid:
            cur_words.append(word)
            before = length_to_index[lens[index]]
            length_to_index[lens[index]] = i + 1
            frequency_pruning(freqs, lens, length_to_words, index + 1, pruned_list, cur_words, length_to_index)
            length_to_index[lens[index]] = before
            cur_words.remove(word)
        for char in word:
            freqs[ord(char) - ord('A')] += 1

def non_overlapping_solution(word_list, word_to_location, index, prev_set, solution, solutions):
    if index >= len(word_list):
        new_sol = []
        for s in solution:
            new_sol.append(s)
        solutions.append(new_sol)
        return
    
    for locations in word_to_location[word_list[index]]:
        new_set = prev_set.union(set(locations))
        if len(new_set) is len(prev_set) + len(locations):
            solution.append(locations)
            non_overlapping_solution(word_list, word_to_location, index + 1, new_set, solution, solutions)
            solution.remove(locations)
            
def main():
    start = time.time()
    lens = []
    lens_set = set()
    board = []
    
    # O(K + RC)
    with open("wordB.in", "r") as f:
        x = [int(i) for i in f.readline().split()]
        R = x[0]
        C = x[1]
        K = x[2]

        lens = [int(i) for i in f.readline().split()]
        for i in lens:
            lens_set.add(i)

        for line in f.readlines():
            temp = []
            my_string = line.rstrip("\n")

            for j in my_string:
                temp.append(j)
            board.append(temp)
    
    
    lens.sort(reverse=True)
    pruned_list = prune_list(board, lens)
    
    word_to_location = {}
    for word in pruned_list:
        find_word(board, word, word_to_location)
    length_to_words = get_length_map(word_to_location)
    freqs = []
    for i in range(26):
        freqs.append(0)
    for row in board:
        for char in row:
            if (char != ' '):
                freqs[ord(char) - ord('A')] += 1
    pruned_list = []
    length_to_index = {}
    for length in lens_set:
        length_to_index[length] = 0
    frequency_pruning(freqs, lens, length_to_words, 0, pruned_list, [], length_to_index)
    end = time.time()
    solutions = []
    word_solutions = []
    for p in pruned_list:
        cur_len = len(solutions)
        non_overlapping_solution(p, word_to_location, 0, set(), [], solutions)
        if len(solutions) > cur_len:
            word_solutions.append(p)
    for w_sol in word_solutions:
        print(w_sol)
    print(len(solutions))
    for s in solutions:
        print(s)
    end = time.time()
    print(end-start)
  
if __name__ == '__main__':
    main()

# '''
# 3 3 2
# 4 5
# GAT
# EMS
# SBE

# BEST
# GAMES

# 3 3 2
# 3 6
# FRT
# IUN
# CKY

# 5 5 5
# 4 7 7 3 4
# MENUS
# ABUFE
# GEBBL
# MSWOR
# OEWAD

# WORD
# BUBBLES
# AWESOME
# FUN
# GAME

# 3 3 2
# 4 4
# NSI
#  ET
# TSE

# NEST
# SITE

# 7 7 7
# 5 6 5 5 6 9 8
# ESSLUI 
# NDLFNFL
# NKYEIIE
# BIATK R
# YEEU ET
# ABTCNUS
# CHOU  G

# YACHT
# BOUNCE
# GUEST
# INFER
# LIKELY
# BEAUTIFUL
# KINDNESS

# 7 7 7
# 5 6 8 7 6 8 4
#   AKBMA
#  LRRSEN
#   AREER
# MMEUSGC
# EOOCTTY
# RMTISDU
# YSBURNR

# STOIC
# MEMORY
# CEREBRAL
# GESTURE
# STURDY
# MARKSMAN
# BURN

# 3 3 1
# 8
#  NA
# TEL
# GGP

# EGGPLANT

# 7 7 7
# 6 4 5 8 7 5 11
# NT TANN
# IRESIYO
# KSENGNT
# EEUSKHI
# TLROCER
# GONPY T
# RUMLIF 

# GRUMPY
# LIFT
# ANNOY
# SKELETON
# THINKER
# SCOUR
# INTERESTING

# 7 7 7
# 8 5 4 7 6 6 8
#  EHAHMO
#  TIERTN
# NFINCYO
# IIMALIO
# HATE DT
# RPO EGA
# GCDRR N

# INFINITE
# TOOTH
# CORD
# HARMONY
# MALICE
# DANGER
# GRAPHITE

# 7 7 7
# 5 9 4 9 10 5 5
#  ECIOSL
# EIFIKKI
# NLUFBVC
# ATCAISI
# ETANGBO
# TEAMERD
#  MMIROE
 
# KIOSK
# EFFICIENT
# TEAM
# ABSORBING
# IMMACULATE
# CIVIL
# ERODE

# 7 7 7
# 5 9 4 9 10 5 5
#  MICFMH
# ILTSAUI
# EAENAML
# US DFIE
# GIZEEAE
# OLAETTH
# VFRVOUC

# FLARE
# FACSIMILE
# MEET
# FANTASIZE
# HUMILIATED
# VOUCH
# VOGUE

# 7 7 7
# 5 9 4 9 10 5 5
# RESWENS
# ETLAPAC
# AT DLTN
# PSSMIHI
# GEC TRE
# ENAEOCK
# RPNRMAS

# ADMIT
# PASSENGER
# THIN
# PACEMAKER
# NEWSLETTER
# SCORN
# SCALP

# 7 7 7
# 5 9 4 9 10 5 5
# YSTIJVE
# AMLNEUG
#  TMAARD
# RERTTER
# YYOJROG
# G OAUHI
# OLRTSWE

# JUDGE
# ASTROLOGY
# RATE
# ASYMMETRY
# VENTILATOR
# JUROR
# WEIGH

# 7 7 5
# 10 10 10 9 9
# MINMJUX
# NSAITPT
# TELISOA
# MELONTS
# SIAMIEA
# AUFAPUT
# MEGURD 

# AMPUTATION
# MINIMALIST
# AMUSEMENTS
# JUXTAPOSE
# LIFEGUARD

# 5 5 4
# 6 6 6 6
# OMEGE
# BILIR
#  EEAO
# RRVND
# PSIUQ

# SPREAD
# REGION
# QUIVER
# MOBILE
# '''

['FACSIMILED', 'FANTASIZE', 'HUMILIATE', 'FLARE', 'VOGUE', 'VOUCH', 'MEET']
['FACSIMILED', 'FANTASIZE', 'HUMILIATE', 'FLARE', 'VOGUE', 'VOUCH', 'TEEM']
['FANTASIZED', 'FACSIMILE', 'HUMILIATE', 'FLARE', 'VOGUE', 'VOUCH', 'MEET']
['FANTASIZED', 'FACSIMILE', 'HUMILIATE', 'FLARE', 'VOGUE', 'VOUCH', 'TEEM']
['HUMILIATED', 'FACSIMILE', 'FANTASIZE', 'FLARE', 'VOGUE', 'VOUCH', 'MEET']
['HUMILIATED', 'FACSIMILE', 'FANTASIZE', 'FLARE', 'VOGUE', 'VOUCH', 'TEEM']
6
[[(0, 4), (1, 4), (0, 3), (1, 3), (0, 2), (0, 1), (1, 0), (1, 1), (2, 2), (3, 3)], [(3, 4), (2, 4), (2, 3), (1, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3)], [(0, 6), (1, 5), (0, 5), (1, 6), (2, 6), (3, 5), (4, 5), (5, 4), (4, 4)], [(6, 1), (5, 1), (5, 2), (6, 2), (5, 3)], [(6, 0), (5, 0), (4, 0), (3, 0), (2, 0)], [(6, 3), (6, 4), (6, 5), (6, 6), (5, 6)], [(2, 5), (3, 6), (4, 6), (5, 5)]]
[[(0, 4), (1, 4), (0, 3), (1, 3), (0, 2), (0, 1), (1, 0), (1, 1), (2, 2), (3, 3)], [(3, 4), (2, 4), (2, 3), (1, 2), (2, 1), (3, 1), (4, 1), (4, 2), (4,